# 🇻🇳 Vietnamese GEC with Contrastive Learning - Google Colab

**Clean & Simple**: Clone repository and run training pipeline for Vietnamese Grammatical Error Correction with BARTpho/ViT5 + Contrastive Learning.

## 📋 Pipeline Overview:
1. **Setup & Clone Repository** - Install dependencies and clone source code
2. **Data Preparation** - Load and preprocess viGEC dataset  
3. **Base Model Training** - Fine-tune BARTpho/ViT5 with hyperparameter optimization
4. **Negative Sample Generation** - Generate negative samples for contrastive learning
5. **Contrastive Learning Training** - Train with contrastive loss + R-Drop
6. **Inference & Evaluation** - Test and evaluate the model

⏰ **Estimated Total Time**: 4-9 hours (depending on GPU)  
🚀 **Ready to Run**: All import issues fixed, clean codebase

## 🚀 Step 1: Setup and Clone Repository

In [ ]:
# Check GPU availability
import torch
print(f"🔥 CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU available - training will be very slow!")

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install numpy
!pip3 install torch torchaudio torchvision torchtext torchdata
!pip install transformers datasets accelerate
!pip install optuna  wandb lightning
!pip install sentencepiece tokenizers nltk sacrebleu evaluate rouge-score
!pip install pandas scikit-learn tqdm rich omegaconf hydra-core
!pip install underthesea pyvi ipywidgets matplotlib seaborn
!pip install -U datasets huggingface_hub fsspec
!pip install optuna-integration[pytorch_lightning]

print("✅ All packages installed successfully!")

In [ ]:
# Clone the repository (replace with your actual GitHub repository URL)
import os

# Change this to your actual repository URL
REPO_URL = "https://github.com/YOUR_USERNAME/CL_GEC.git"  # Update this!
PROJECT_DIR = "/content/CL_GEC"

# Clone or update repository
if not os.path.exists(PROJECT_DIR):
    print(f"📥 Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL} {PROJECT_DIR}
else:
    print("📁 Repository already exists, pulling latest changes...")
    %cd {PROJECT_DIR}
    !git pull

# Change to project directory
%cd {PROJECT_DIR}
print(f"📂 Working directory: {os.getcwd()}")

# List files to verify
print("\n📋 Project files:")
!ls -la *.py

## 📊 Step 2: Data Preparation and System Check

In [ ]:
# Test imports and system readiness
from data_utils import (
    load_vigec_dataset, 
    get_model_and_tokenizer, 
    can_train_base_model,
    check_dataset_format
)
from rich.console import Console

console = Console()

# Check system readiness
console.print("[bold blue]🔍 System Readiness Check[/bold blue]")
system_ready = can_train_base_model()

# Check dataset format
console.print("\n[bold blue]📋 Dataset Format Check[/bold blue]")
dataset_ready = check_dataset_format()

if system_ready and dataset_ready:
    console.print("\n[bold green]✅ All checks passed! Ready to proceed.[/bold green]")
else:
    console.print("\n[bold red]❌ System not ready. Please check requirements.[/bold red]")

In [ ]:
# Load and prepare dataset
console.print("[bold blue]📊 Loading viGEC Dataset[/bold blue]")

# Load dataset (using small subset for faster processing in Colab)
data = load_vigec_dataset(
    dataset_name="phuhuy-se1/viGEC",
    test_subset_ratio=0.05  # Use 5% of test set for faster evaluation
)

console.print(f"\n[green]Dataset loaded successfully![/green]")
for split, split_data in data.items():
    console.print(f"  {split}: {len(split_data)} samples")

# Save processed data
from data_utils import save_processed_data
save_processed_data(data, "./data/processed")
console.print("\n[blue]✅ Data saved to ./data/processed/[/blue]")

## 🤖 Step 3: Model Selection and Testing

In [ ]:
# Choose your model - uncomment one of these:
MODEL_NAME = "vinai/bartpho-syllable"  # Recommended for Vietnamese
# MODEL_NAME = "VietAI/vit5-base"     # Alternative option
# MODEL_NAME = "VietAI/vit5-large"    # Larger model (requires more GPU memory)

console.print(f"[bold blue]🤖 Loading Model: {MODEL_NAME}[/bold blue]")

# Load model and tokenizer
model, tokenizer = get_model_and_tokenizer(MODEL_NAME)

console.print(f"[green]✅ Model loaded successfully![/green]")
console.print(f"  Model: {model.__class__.__name__}")
console.print(f"  Tokenizer: {tokenizer.__class__.__name__}")
console.print(f"  Vocabulary size: {len(tokenizer)}")

# Test tokenization
test_text = "Tôi đang học tiếng việt."
tokens = tokenizer(test_text, return_tensors="pt")
console.print(f"\n[blue]🧪 Tokenization Test:[/blue]")
console.print(f"  Input: {test_text}")
console.print(f"  Tokens: {tokens['input_ids'].shape}")

## 🏋️ Step 4: Base Model Training

In [ ]:
# Configure training parameters
TRAINING_CONFIG = {
    "model_name": MODEL_NAME,
    "output_dir": "./models/base_model",
    "max_epochs": 3,  # Reduced for Colab
    "batch_size": 8,  # Adjust based on GPU memory
    "learning_rate": 5e-5,
    "use_wandb": True,  # Set to False if you don't want to use Weights & Biases
    "run_optimization": False,  # Set to True for hyperparameter optimization (takes longer)
}

console.print("[bold blue]🏋️ Base Model Training Configuration:[/bold blue]")
for key, value in TRAINING_CONFIG.items():
    console.print(f"  {key}: {value}")

In [ ]:
# Start base model training
from base_trainer import BaseTrainer

console.print("[bold green]🚀 Starting Base Model Training...[/bold green]")

# Create base trainer
base_trainer = BaseTrainer(
    model_name=TRAINING_CONFIG["model_name"],
    output_dir=TRAINING_CONFIG["output_dir"],
    use_wandb=TRAINING_CONFIG["use_wandb"]
)

# Train the model
if TRAINING_CONFIG["run_optimization"]:
    console.print("[yellow]🔍 Running hyperparameter optimization...[/yellow]")
    best_params = base_trainer.optimize_hyperparameters(
        data=data,
        n_trials=10,  # Reduced for Colab
        timeout=3600  # 1 hour timeout
    )
    console.print(f"[green]✅ Best parameters: {best_params}[/green]")
else:
    console.print("[blue]🏃 Training with default parameters...[/blue]")
    base_trainer.train(
        data=data,
        max_epochs=TRAINING_CONFIG["max_epochs"],
        batch_size=TRAINING_CONFIG["batch_size"],
        learning_rate=TRAINING_CONFIG["learning_rate"]
    )

console.print("[bold green]✅ Base model training completed![/bold green]")

## 🎯 Step 5: Negative Sample Generation

In [ ]:
# Generate negative samples for contrastive learning
from negative_sampler import NegativeSampler

console.print("[bold blue]🎯 Generating Negative Samples...[/bold blue]")

# Create negative sampler
negative_sampler = NegativeSampler(
    model_path=TRAINING_CONFIG["output_dir"],  # Use trained base model
    model_name=MODEL_NAME
)

# Generate negative samples for training data
contrastive_data = negative_sampler.generate_contrastive_dataset(
    data=data['train'][:1000],  # Use subset for faster processing in Colab
    num_negatives=3,  # Generate 3 negative samples per positive
    output_file="./data/contrastive_train.json"
)

console.print(f"[green]✅ Generated {len(contrastive_data)} contrastive samples![/green]")
console.print("[blue]💾 Saved to ./data/contrastive_train.json[/blue]")

## 🔥 Step 6: Contrastive Learning Training

In [ ]:
# Contrastive learning training
from contrastive_trainer import ContrastiveTrainer

console.print("[bold blue]🔥 Starting Contrastive Learning Training...[/bold blue]")

# Create contrastive trainer
contrastive_trainer = ContrastiveTrainer(
    base_model_path=TRAINING_CONFIG["output_dir"],
    model_name=MODEL_NAME,
    output_dir="./models/contrastive_model",
    use_wandb=TRAINING_CONFIG["use_wandb"]
)

# Train with contrastive learning
contrastive_trainer.train(
    contrastive_data=contrastive_data,
    validation_data=data['validation'][:200],  # Use subset for validation
    max_epochs=2,  # Fewer epochs for contrastive learning
    batch_size=4,  # Smaller batch size due to multiple samples per item
    learning_rate=2e-5,  # Lower learning rate for fine-tuning
    contrastive_weight=0.5,  # Weight for contrastive loss
    rdrop_weight=0.1  # Weight for R-Drop regularization
)

console.print("[bold green]✅ Contrastive learning training completed![/bold green]")

## 🧪 Step 7: Inference and Evaluation

In [ ]:
# Load the best model for inference
from inference import GECInference

console.print("[bold blue]🧪 Setting up Inference...[/bold blue]")

# Create inference engine with the best model
gec_inference = GECInference(
    model_path="./models/contrastive_model",  # Use contrastive model
    model_name=MODEL_NAME
)

console.print("[green]✅ Inference engine ready![/green]")

In [ ]:
# Interactive testing
console.print("[bold blue]🎮 Interactive Testing[/bold blue]")

# Test samples
test_sentences = [
    "Tôi đang học tiếng việt ở trường đại học.",
    "Hôm nay trời rất đẹp và tôi muốn đi chơi.",
    "Cô ấy làm việc tại một công ty lớn ở Hà Nội.",
    "Chúng tôi sẽ đi du lịch vào cuối tuần này."
]

console.print("\n[yellow]📝 Test Results:[/yellow]")
for i, sentence in enumerate(test_sentences, 1):
    corrected = gec_inference.correct_text(sentence)
    console.print(f"\n{i}. Original: {sentence}")
    console.print(f"   Corrected: {corrected}")

# Custom input
console.print("\n[bold cyan]✏️ Try your own text:[/bold cyan]")
print("Enter Vietnamese text to correct (or 'quit' to exit):")

while True:
    user_input = input("> ")
    if user_input.lower() == 'quit':
        break
    
    corrected = gec_inference.correct_text(user_input)
    print(f"Corrected: {corrected}\n")

In [ ]:
# Evaluate on test set
from evaluator import F05Evaluator

console.print("[bold blue]📊 Evaluating on Test Set...[/bold blue]")

# Create evaluator
evaluator = F05Evaluator()

# Evaluate on test set (using subset for faster evaluation)
test_data = data['test'][:100]  # Use 100 samples for evaluation
sources = [item['source'] for item in test_data]
references = [item['target'] for item in test_data]

# Generate predictions
console.print("[yellow]🔮 Generating predictions...[/yellow]")
predictions = []
for source in sources:
    pred = gec_inference.correct_text(source)
    predictions.append(pred)

# Calculate metrics
results = evaluator.evaluate_batch(predictions, references, sources)

console.print("\n[bold green]📈 Evaluation Results:[/bold green]")
for metric, value in results.items():
    if isinstance(value, float):
        console.print(f"  {metric}: {value:.4f}")
    else:
        console.print(f"  {metric}: {value}")

## 💾 Step 8: Save and Export Results

In [ ]:
# Save results and create export package
import json
import zipfile
from datetime import datetime

console.print("[bold blue]💾 Saving Results and Creating Export Package...[/bold blue]")

# Create results summary
results_summary = {
    "timestamp": datetime.now().isoformat(),
    "model_name": MODEL_NAME,
    "training_config": TRAINING_CONFIG,
    "evaluation_results": results,
    "test_samples": len(test_data),
    "model_paths": {
        "base_model": TRAINING_CONFIG["output_dir"],
        "contrastive_model": "./models/contrastive_model"
    }
}

# Save results
with open("./results_summary.json", "w", encoding="utf-8") as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)

console.print("[green]✅ Results saved to ./results_summary.json[/green]")

# Create downloadable package
console.print("[yellow]📦 Creating export package...[/yellow]")

with zipfile.ZipFile("vietnamese_gec_models.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    # Add results
    zipf.write("results_summary.json")
    
    # Add model files (if they exist)
    import glob
    for model_file in glob.glob("./models/**/*.bin", recursive=True):
        zipf.write(model_file)
    for config_file in glob.glob("./models/**/config.json", recursive=True):
        zipf.write(config_file)
    
    # Add data samples
    if os.path.exists("./data/contrastive_train.json"):
        zipf.write("./data/contrastive_train.json")

console.print("[bold green]🎉 Export package created: vietnamese_gec_models.zip[/bold green]")
console.print("[blue]📁 You can download this file from the Colab file browser[/blue]")

# Display final summary
console.print("\n[bold cyan]🏆 Training Pipeline Completed Successfully![/bold cyan]")
console.print(f"[green]✅ Base model trained and saved[/green]")
console.print(f"[green]✅ Contrastive learning applied[/green]")
console.print(f"[green]✅ Model evaluated on test set[/green]")
console.print(f"[green]✅ Results exported for download[/green]")